### Activation Functions

In [1]:
def sigmoid(x):
    return 1./(1+np.exp(-x))

def relu(x):
    x[x<0.0] = 0.0
    return x

def deriv_relu(x):
    x[x>0.0] = 1.0
    return x

def tanh(x):
    return np.tanh(x)

def deriv_tanh(x):
    return (1-x**2)

### Import Necessary Packages (load MNIST data and preprocess it)

In [2]:
import numpy as np
np.random.seed(1234)
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
mnist = fetch_mldata('MNIST original')
# prepare data
N = 70000
data = np.float32(mnist.data[:]) / 255.
idx = np.random.choice(data.shape[0], N)
data = data[idx]
target = np.int32(mnist.target[idx]).reshape(N, 1)

train_idx, test_idx = train_test_split(np.array(range(N)), test_size=0.10)
train_data, test_data = data[train_idx], data[test_idx]
train_target, test_target = target[train_idx], target[test_idx]

train_target = np.float32(preprocessing.OneHotEncoder(sparse=False).fit_transform(train_target))

# inputs
n_input = train_data.shape[1]
M = train_data.shape[0]

batch_size = 100 # 4
num_batches = train_data.shape[0]//batch_size

/Users/thakur/newenv/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Setting the sigma prior, initializing the mu, sigma values

In [3]:
sigma_prior = np.exp(-3)
learning_rate = 0.001 # from 0.001, changed from 0.0005 of linear regression
n_epochs = 50
n_hidden_1 = 400
n_hidden_2 = 400
n_output = 10

batch_size = 100
n_batches = int(M / float(batch_size))

mu_1 = np.random.normal(0, 0.05, size=(n_input, n_hidden_1))
sigma_1 = np.random.normal(0, 0.05, size=(n_input, n_hidden_1))
mu_2 = np.random.normal(0, 0.05, size=(n_hidden_1, n_hidden_2))
sigma_2 = np.random.normal(0, 0.05, size=(n_hidden_1, n_hidden_2))
mu_3 = np.random.normal(0, 0.05, size=(n_hidden_2, n_output))
sigma_3 = np.random.normal(0, 0.05, size=(n_hidden_2, n_output))
b_b1_mu = np.random.normal(0, 0.05, size=(1, n_hidden_1))
b_b1_logsigma = np.random.normal(0, 0.05, size=(1, n_hidden_1))
b_b2_mu = np.random.normal(0, 0.05, size=(1, n_hidden_2))
b_b2_logsigma = np.random.normal(0, 0.05, size=(1, n_hidden_2))
b_b3_mu = np.random.normal(0, 0.05, size=(1, n_output))
b_b3_logsigma = np.random.normal(0, 0.05, size=(1, n_output))

### This is for forward and back propagtion

In [4]:
def forward_backprop(mu_1, sigma_1, eps_1, mu_2, sigma_2, eps_2, mu_3, sigma_3, eps_3, 
    b_b1_mu, b_b1_logsigma, b_epsilon_b1,b_b2_mu, b_b2_logsigma, b_epsilon_b2,
    b_b3_mu, b_b3_logsigma, b_epsilon_b3, x_in, y_output, batch_size, n_batches, lr, b, episode_num):
    weight_1 = mu_1 + np.log(1. + np.exp(sigma_1))*eps_1
    weight_2 = mu_2 + np.log(1. + np.exp(sigma_2))*eps_2
    weight_3 = mu_3 + np.log(1. + np.exp(sigma_3))*eps_3
    b_weight_1 = b_b1_mu + np.log(1+np.exp(b_b1_logsigma))*b_epsilon_b1
    b_weight_2 = b_b2_mu + np.log(1+np.exp(b_b2_logsigma))*b_epsilon_b2
    b_weight_3 = b_b3_mu + np.log(1+np.exp(b_b3_logsigma))*b_epsilon_b3
    z1 = np.dot(x_in, weight_1) + b_weight_1
    h1 = relu(z1)
    z2 = np.dot(h1, weight_2) + b_weight_2 
    h2 = relu(z2)
    z3 = np.dot(h2, weight_3) + b_weight_3
    z3 = np.exp(z3)/np.sum(np.exp(z3), axis=1, keepdims=True)

    
    # likelihood
    delta3 = -1.0*(1./x_in.shape[0])*(y_output-z3)/(np.exp(-3)**2) # we need to check (y-h) or (h-y)
    delta_b3_mu = np.sum(delta3, axis=0)
    delta_b3_logsigma = np.sum((delta3)*(1/(1+np.exp(-b_b3_logsigma)))*b_epsilon_b3, axis=0)
    delta_w3_mu = np.dot(h2.T, delta3)
    delta_w3_logsigma = np.dot(h2.T, delta3)*(1/(1+np.exp(-sigma_3)))*eps_3
    delta2 = np.dot(delta3, weight_3.T)*deriv_relu(h2)
    delta_b2_mu = np.sum(delta2, axis=0)
    delta_b2_logsigma = np.sum((delta2)*(1/(1+np.exp(-b_b2_logsigma)))*b_epsilon_b2, axis=0)
    delta_w2_mu = np.dot(h1.T, delta2)
    delta_w2_logsigma = np.dot(h1.T, delta2)*(1/(1+np.exp(-sigma_2)))*eps_2
    delta1 = np.dot(delta2, weight_2.T)*deriv_relu(h1)
    delta_b1_mu = np.sum(delta1, axis=0)
    delta_b1_logsigma = np.sum((delta1)*(1/(1+np.exp(-b_b1_logsigma)))*b_epsilon_b1, axis=0)
    delta_w1_mu = np.dot(x_in.T, delta1)
    delta_w1_logsigma = np.dot(x_in.T, delta1)*(1/(1+np.exp(-sigma_1)))*eps_1
    
    # prior 
    # wegihts
    w_gaussian_prior_mu_1 = -weight_1/(0.05**2)
    w_gaussian_prior_sigma_1 = (-weight_1/(0.05**2))* eps_1*(1./(1.+np.exp(-sigma_1)))
    w_gaussian_prior_mu_2 = -weight_2/(0.05**2)
    w_gaussian_prior_sigma_2 = (-weight_2/(0.05**2))* eps_2*(1./(1.+np.exp(-sigma_2)))
    w_gaussian_prior_mu_3 = -weight_3/(0.05**2)
    w_gaussian_prior_sigma_3 = (-weight_3/(0.05**2))* eps_3*(1./(1.+np.exp(-sigma_3)))
    # biases
    b_gaussian_prior_mu_1 = -b_weight_1/(0.05**2)
    b_gaussian_prior_sigma_1 = (-b_weight_1/(0.05**2))* b_epsilon_b1*(1./(1.+np.exp(-b_b1_logsigma)))
    b_gaussian_prior_mu_2 = -b_weight_2/(0.05**2)
    b_gaussian_prior_sigma_2 = (-b_weight_2/(0.05**2))* b_epsilon_b2*(1./(1.+np.exp(-b_b2_logsigma)))
    b_gaussian_prior_mu_3 = -b_weight_3/(0.05**2)
    b_gaussian_prior_sigma_3 = (-b_weight_3/(0.05**2))* b_epsilon_b3*(1./(1.+np.exp(-b_b3_logsigma)))
    
    # variational posterior only sigmas
    # sigmas for weights (mus for weights is zero in the case of Variational Posterior)
    varitional_sigma_1 = (-1.0/(np.log(1. + np.exp(sigma_1))))*(1/(1+np.exp(-sigma_1)))
    varitional_sigma_2 = (-1.0/(np.log(1. + np.exp(sigma_2))))*(1/(1+np.exp(-sigma_2)))
    varitional_sigma_3 = (-1.0/(np.log(1. + np.exp(sigma_3))))*(1/(1+np.exp(-sigma_3)))
    
    # sigmas for bias terms
    b_varitional_sigma_1 = (-1.0/(np.log(1+np.exp(b_b1_logsigma))))*(1/(1+np.exp(-b_b1_logsigma)))
    b_varitional_sigma_2 = (-1.0/(np.log(1+np.exp(b_b2_logsigma))))*(1/(1+np.exp(-b_b2_logsigma)))
    b_varitional_sigma_3 = (-1.0/(np.log(1+np.exp(b_b3_logsigma))))*(1/(1+np.exp(-b_b3_logsigma)))

    # mus and sigmas updates
    mu_1 = mu_1 - lr*((1./n_batches)*(0-w_gaussian_prior_mu_1)+(delta_w1_mu*x_in.shape[0]))*(1./x_in.shape[0])
    sigma_1 = sigma_1 - lr*((1./n_batches)*(varitional_sigma_1-w_gaussian_prior_sigma_1) + \
        (delta_w1_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    b_b1_mu = b_b1_mu - lr*((1./n_batches)*(0-b_gaussian_prior_mu_1)+(delta_b1_mu*x_in.shape[0]))*(1./x_in.shape[0])
    b_b1_logsigma = b_b1_logsigma - lr*((1./n_batches)*(b_varitional_sigma_1-b_gaussian_prior_sigma_1) + \
        (delta_b1_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    mu_2 = mu_2 - lr*((1./n_batches)*(0-w_gaussian_prior_mu_2)+ (delta_w2_mu*x_in.shape[0]))*(1./x_in.shape[0])
    sigma_2 = sigma_2 - lr*((1./n_batches)*(varitional_sigma_2-b_gaussian_prior_sigma_2) + \
        (delta_w2_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    b_b2_mu = b_b2_mu - lr*((1./n_batches)*(0-b_gaussian_prior_mu_2) + \
        (delta_b2_mu*x_in.shape[0]))*(1./x_in.shape[0])
    b_b2_logsigma = b_b2_logsigma - lr*((1./n_batches)*(b_varitional_sigma_2-b_gaussian_prior_sigma_2) + \
        (delta_b2_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    mu_3 = mu_3 - lr*((1./n_batches)*(0-w_gaussian_prior_mu_3)+(delta_w3_mu*x_in.shape[0]))*(1./x_in.shape[0])
    sigma_3 = sigma_3 - lr*((1./n_batches)*(varitional_sigma_3-w_gaussian_prior_sigma_3) + \
        (delta_w3_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    b_b3_mu = b_b3_mu - lr*((1./n_batches)*(0-b_gaussian_prior_mu_3)+(delta_b3_mu*x_in.shape[0]))*(1./x_in.shape[0])
    b_b3_logsigma = b_b3_logsigma - lr*((1./n_batches)*(b_varitional_sigma_3-b_gaussian_prior_sigma_3) + \
        (delta_b3_logsigma*x_in.shape[0]))*(1./x_in.shape[0])

    # this method returns the new mus and sigmas after updating
    return (mu_1, sigma_1, mu_2, sigma_2, mu_3, sigma_3, b_b1_mu, b_b1_logsigma, b_b2_mu, b_b2_logsigma, b_b3_mu, b_b3_logsigma)


### Training

In [ ]:
for e in range(n_epochs):
    test_accuracy = []
    train_accuracy = []

    for b in range(n_batches):
        x_input = np.reshape(train_data[b*batch_size:(b+1)*batch_size,:], (batch_size,784))
        y_true = np.reshape(train_target[b*batch_size:(b+1)*batch_size,:],(batch_size,10))
        eps_1 = np.random.normal(0, 0.05, size=(n_input, n_hidden_1))
        eps_2 = np.random.normal(0, 0.05, size=(n_hidden_1, n_hidden_2))
        eps_3 = np.random.normal(0, 0.05, size=(n_hidden_2, n_output))
        b_epsilon_b1 = np.random.normal(0, 0.05, size=(1, n_hidden_1))
        b_epsilon_b2 = np.random.normal(0, 0.05, size=(1, n_hidden_2))
        b_epsilon_b3 = np.random.normal(0, 0.05, size=(1, n_output))
        mu_1, sigma_1, mu_2, sigma_2, mu_3, sigma_3, b_b1_mu, b_b1_logsigma, b_b2_mu, b_b2_logsigma, b_b3_mu, \
        b_b3_logsigma = forward_backprop(mu_1, sigma_1, eps_1, mu_2, sigma_2, eps_2, mu_3, sigma_3, eps_3, b_b1_mu, \
                               b_b1_logsigma, b_epsilon_b1, b_b2_mu, b_b2_logsigma, b_epsilon_b2, b_b3_mu, b_b3_logsigma, b_epsilon_b3, x_input, y_true, batch_size, n_batches, learning_rate, b, e)

    if e%1 == 0.0:
        z1 = np.dot(train_data, mu_1) + b_b1_mu
        h1 = relu(z1)
        z2 = np.dot(h1, mu_2) + b_b2_mu 
        h2 = relu(z2)
        z3 = np.dot(h2, mu_3) + b_b3_mu
        out_1 = np.exp(z3)/np.sum(np.exp(z3), axis=1, keepdims=True)      

        soft_arg_max_1 = np.asarray(np.argmax(out_1, axis=1))
        test_numeric_1 = np.argmax(train_target, axis=1)
        acc_1 = np.count_nonzero(soft_arg_max_1 == test_numeric_1)
        #print('episode num', e)
        train_accuracy.append(acc_1/soft_arg_max_1.shape[0])
        
        z1 = np.dot(test_data, mu_1) + b_b1_mu
        h1 = relu(z1)
        z2 = np.dot(h1, mu_2) + b_b2_mu 
        h2 = relu(z2)
        z3 = np.dot(h2, mu_3) + b_b3_mu
        out_2 = np.exp(z3)/np.sum(np.exp(z3), axis=1, keepdims=True)      

        soft_arg_max_2 = np.asarray(np.argmax(out_2, axis=1))
        #test_numeric_2 = np.argmax(test_target, axis=1)
        count = 0

        for i in range(len(test_target)):
            if test_target[i] == soft_arg_max_2[i]:
                count += 1
        print("epoch", e,"test accuracy",(count/len(test_target)),"train accuracy",(acc_1/soft_arg_max_1.shape[0]))
        test_accuracy.append(count/len(test_target))

epoch 0 test accuracy 0.9592857142857143 train accuracy 0.9666190476190476
epoch 1 test accuracy 0.9737142857142858 train accuracy 0.9811587301587301
epoch 2 test accuracy 0.9805714285714285 train accuracy 0.9873650793650793
epoch 3 test accuracy 0.9832857142857143 train accuracy 0.9906507936507937
epoch 4 test accuracy 0.9862857142857143 train accuracy 0.9935079365079366
epoch 5 test accuracy 0.9868571428571429 train accuracy 0.9943333333333333
epoch 6 test accuracy 0.9887142857142858 train accuracy 0.997
epoch 7 test accuracy 0.9897142857142858 train accuracy 0.9974285714285714
epoch 8 test accuracy 0.9887142857142858 train accuracy 0.9969365079365079
epoch 9 test accuracy 0.9907142857142858 train accuracy 0.9987619047619047
epoch 10 test accuracy 0.9912857142857143 train accuracy 0.9986825396825397
epoch 11 test accuracy 0.9907142857142858 train accuracy 0.9991746031746032
epoch 12 test accuracy 0.9918571428571429 train accuracy 0.9991111111111111
epoch 13 test accuracy 0.993 train 